#### Download all images from FTP to local filesystem

In [ ]:
import os

from dotenv import load_dotenv
from ftp import SingletonFTP

load_dotenv()
host = os.getenv('FTP_HOST')
username = os.getenv('FTP_USERNAME')
password = os.getenv('FTP_PASSWORD')


ftp = SingletonFTP(host, username, password)
ftp.download_all('/files/files-mailseth', 'tmp')

### Detect objects in frames

In [ ]:
from object_detection import object_detection
from tqdm import tqdm
import gc

image_dir = 'tmp'


image_files = [os.path.join(image_dir, f)
               for f in os.listdir(image_dir) if f.endswith('.jpg')]
image_files_iter = iter(image_files)

output_list = []

image_files_in_output_list = [output['image_file'] for output in output_list]

for image_file in tqdm(image_files_iter):
    if image_file not in image_files_in_output_list:
        detection_result = object_detection(image_file)
        detection_result_json = detection_result.pandas().xyxy[0].to_json()
        print('image_file', image_file)
        print('detection_result_json', detection_result_json)
        output_list.append({
            'image_file': image_file,
            'detection_result': detection_result_json
        })
        # Explicitly delete the detection_result and call the garbage collector
        del detection_result
        gc.collect()